In [ ]:
'''
Introdução à Econometria - Uma abordagem moderna (Tradução da 6 edição norte-americana)
Autor: WOOLDRIDGE, J. M.
Editora: CENGAGE LEARNING

Cap. 8: Heterocedasticidade (Heteroskedasticity)
Exemplo 8.6: Equação de Patrimônio Financeiro
             (FINANCIAL WEALTH EQUATION)
             
Arquivo com os dados: 401ksubs.xls

Arquivo com dados em:
http://students.cengage.com.br/dashboard/private/livroView.jsf;jsessionid=95E9AD889A4A4B7ABBD2A5251F1E14BE?id=104577

Em caso de dúvidas ou problemas, solicitamos, por gentileza, entrar em contato pelo e-mail:
python.economia@gmail.com
'''

In [1]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

import statsmodels.iolib.summary2 as sis

In [2]:
df = pd.read_excel('401ksubs.xls',
                   header=None,
                   usecols=[0, 1, 3, 4, 5, 6],
                   names=['e401k', 'inc', 'male', 'age', 'fsize', 'nettfa'])

In [3]:
df.head()

,e401k,inc,male,age,fsize,nettfa
0,0,13.170,0,40,1,4.575
1,1,61.230,1,35,1,154.000
2,0,12.858,0,44,2,0.000
3,0,98.880,1,44,2,21.800
4,0,22.614,0,53,1,18.450


In [4]:
filtro = (df['fsize'] == 1)    # Apenas famílias de uma única pessoa
df_filtrado = df[filtro]

df_filtrado.head()

,e401k,inc,male,age,fsize,nettfa
0,0,13.170,0,40,1,4.575
1,1,61.230,1,35,1,154.000
4,0,22.614,0,53,1,18.450
9,1,29.100,1,45,1,29.600
15,0,19.074,1,43,1,0.000


### Solução sugerida

In [5]:
# Regressão pelo Método de Mínimos Quadrados Ordinários (MQO ou OLS na sigla em inglês)
model_1 = smf.ols('nettfa ~ inc', data=df_filtrado)
reg_1 = model_1.fit(cov_type='HC0')    # implementa procedimento robusto em relação à heterocedasticidade

model_3 = smf.ols('nettfa ~ inc + np.square(age - 25) + male + e401k', data=df_filtrado)
reg_3 = model_3.fit(cov_type='HC0')    # implementa procedimento robusto em relação à heterocedasticidade

In [6]:
# Regressão pelo Método de Mínimos Quadrados Ponderados (MQP ou WLS na sigla em inglês)

'''
Hipótese utilizada neste exemplo para execução da regressão pelo método de Mínimos Quadrados Ponderados:
var(u|inc) = sigma_quad * inc    -->  peso = 1 / inc
'''
peso_mqp = 1 / df_filtrado['inc'] 

model_2 = smf.wls('nettfa ~ inc', weights=peso_mqp, data=df_filtrado)
reg_2 = model_2.fit()

model_4 = smf.wls('nettfa ~ inc + np.square(age - 25) + male + e401k', weights=peso_mqp, data=df_filtrado)
reg_4 = model_4.fit()


In [7]:
results_table = sis.summary_col(results=[reg_1,reg_2, reg_3, reg_4],
                            float_format='%.3f',
                            stars = False,
                            model_names=['1\n(MQO)', '2\n(MQP)', '3\n(MQO)', '4\n(MQP)'],
                            regressor_order=['inc', 'np.square(age - 25)', 'male', 'e401k', 'Intercept'],
                            info_dict={"Núm. Obs.:": lambda x: f"{int(x.nobs)}"})

results_table.add_title('Exemplo 8.6 [Variável dependente:nettfa]')
print(results_table)

     Exemplo 8.6 [Variável dependente:nettfa]
                       1       2       3       4   
                     (MQO)   (MQP)   (MQO)   (MQP) 
---------------------------------------------------
inc                 0.821   0.787   0.771   0.740  
                    (0.104) (0.063) (0.099) (0.064)
np.square(age - 25)                 0.025   0.018  
                                    (0.004) (0.002)
male                                2.478   1.841  
                                    (2.056) (1.564)
e401k                               6.886   5.188  
                                    (2.284) (1.703)
Intercept           -10.571 -9.581  -20.985 -16.703
                    (2.529) (1.653) (3.491) (1.958)
R-squared           0.083   0.071   0.128   0.112  
R-squared Adj.      0.082   0.070   0.126   0.110  
Núm. Obs.:          2017    2017    2017    2017   
Standard errors in parentheses.


In [8]:
'''
Estatística-F para o modelo 4 (em que usamos Mínimos Quadrados Ponderados)

H0: B_(age - 25)**2 = 0, B_male = 0, B_e401k = 0
H1: pelo menos um dos parâmetros na hipótese nula é diferente de zero
'''

hipotese = 'np.square(age - 25) = 0, male = 0, e401k = 0'
teste_f = reg_4.f_test(hipotese)
print(f"Estatística F: {teste_f.fvalue[0]}")
print(f"Valor p: {teste_f.pvalue}")

Estatística F: [30.66851822]
Valor p: 2.205573511773886e-19


In [ ]:
'''
Conclusão teste F

O valor-p obtido é praticamente igual a zero (note que o valor de 2.2 é multiplicado por 10 elevado a -19). 
Assim temos uma forte evidência contra H0.
'''